from sagemaker.sklearn.model import SKLearnModel
import sagemaker

# SageMaker variables
role = sagemaker.get_execution_role()
output_path = f"s3://your-bucket-name/batch-transform-output/"
instance_type = "ml.m5.large"

# Get model artifact S3 URI from Model Registry
sm_client = boto3.client('sagemaker', region_name='your-region')
response = sm_client.describe_model_package(
    ModelPackageGroupName="your-model-package-group-name",  # Update with your Model Package Group Name
    ModelPackageVersion=int(model_version)  # Update with your Model Version
)
model_tar_s3_uri = response['InferenceSpecification']['Containers'][0]['ModelDataUrl']
print(f"Model artifact S3 URI: {model_tar_s3_uri}")

# Create SKLearnModel with the retrieved model.tar.gz URI
sklearn_model = SKLearnModel(
    model_data=model_tar_s3_uri,
    role=role,
    entry_point="inference.py",
    framework_version="0.23-1",  # Match your scikit-learn version
)

# Deploy the model to batch transform
transformer = sklearn_model.transformer(
    instance_count=1,
    instance_type=instance_type,
    output_path=output_path,
    assemble_with="Line",
    accept="text/csv",
)

# Start the batch transform job
transformer.transform(
    data="s3://your-bucket-name/test.csv",
    content_type="text/csv",
    split_type="Line",
    input_filter="$[1:]",  # Update based on your input needs
)

# Wait for job completion
transformer.wait()

print(f"Batch Transform Job completed. Output saved to {output_path}")
